In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import math

In [3]:
import os
os.chdir(r"C:\Users\sharo\Documents\Postgrad\My Data Science Portfolio\Transforming the Way We Search For Flats")

## Import Resale Data
- data frm Data.gov.sg
- transactions from Jan 2017-20 Sep 2021

In [4]:
resale = pd.read_csv("HDB Resale Prices/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv",
                    usecols=['block','street_name','flat_type','storey_range','floor_area_sqm','resale_price'])

resale.rename(columns={'block':'blk_no','street_name':'street'},inplace=True)

resale.head()

,flat_type,blk_no,street,storey_range,floor_area_sqm,resale_price
0,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0
1,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0
2,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0
3,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0
4,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0


## Import clean resi-blks dataset

In [5]:
resi_blks = pd.read_csv("Clean Datasets/resi_blks2.csv")
resi_blks=resi_blks.drop(columns=['max_floor_lvl','bldg_contract_town','total_dwelling_units','1room_rental', 
                                  '2room_rental', '3room_rental','other_room_rental', 'n_sold', 
                                  'n_rental', 'n_residents','remaining_lease', 'Latitude', 'Longitude','hdb_town'])
resi_blks.head()

,blk_no,street,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,Postcode
0,1,BEACH RD,0,1,138,1,2,0,0,0,190001
1,1,BEDOK STH AVE 1,0,0,204,0,2,0,0,0,460001
2,1,CHAI CHEE RD,0,0,0,10,92,0,0,0,461001
3,1,CHANGI VILLAGE RD,0,0,54,0,1,0,0,0,500001
4,1,DELTA AVE,0,0,0,0,96,0,0,0,160001


## Add postcodes to resale data

In [6]:
postcode_lookup = resi_blks.loc[:,['blk_no','street','Postcode']]
postcode_lookup.head()

,blk_no,street,Postcode
0,1,BEACH RD,190001
1,1,BEDOK STH AVE 1,460001
2,1,CHAI CHEE RD,461001
3,1,CHANGI VILLAGE RD,500001
4,1,DELTA AVE,160001


In [7]:
resale_1 = resale.merge(postcode_lookup, how='left', on=['blk_no','street'])
# resale_1['Postcode'] = resale_1['Postcode'].apply(int)

resale_1.head()

,flat_type,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode
0,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,560406.0
1,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,560108.0
2,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,560602.0
3,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,560465.0
4,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,560601.0


In [8]:
#extract the ones which could not be matched
resale_1_nulls = resale_1[resale_1['Postcode'].isnull()]
resale_1_nulls

,flat_type,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode
632,3 ROOM,3,ST. GEORGE'S RD,13 TO 15,59.0,286000.0,NaN
634,3 ROOM,3,ST. GEORGE'S RD,07 TO 09,59.0,320000.0,NaN
658,5 ROOM,21,ST. GEORGE'S RD,07 TO 09,118.0,599000.0,NaN
1458,4 ROOM,216,CHOA CHU KANG CTRL,01 TO 03,108.0,345000.0,NaN
1708,3 ROOM,3,ST. GEORGE'S RD,10 TO 12,59.0,305000.0,NaN
...,...,...,...,...,...,...,...
105184,4 ROOM,4B,ST. GEORGE'S LANE,01 TO 03,105.0,570000.0,NaN
105185,4 ROOM,9,ST. GEORGE'S RD,04 TO 06,84.0,425000.0,NaN
107411,3 ROOM,3,ST. GEORGE'S RD,13 TO 15,59.0,295000.0,NaN
107412,3 ROOM,3,ST. GEORGE'S RD,13 TO 15,59.0,306000.0,NaN


In [9]:
#fix streetnames in different formats which hinder matching. these were the same ones which we had to fix while doing geocoding for resi-blks

#noticed that almost all the null entries were at St. George's Road. On OneMap website, Saint is spelt out in full. 
resale_1_nulls['street'] = resale_1_nulls['street'].str.replace(pat="ST.", repl="SAINT",case=False,regex=False)

#For the remainin blocks, CENTRAL is spelled as CTRL
resale_1_nulls['street'] = resale_1_nulls['street'].str.replace(pat="CTRL", repl="CENTRAL",case=False,regex=False)

C:\Users\sharo\AppData\Local\Temp/ipykernel_26280/3568192099.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resale_1_nulls['street'] = resale_1_nulls['street'].str.replace(pat="ST.", repl="SAINT",case=False,regex=False)
C:\Users\sharo\AppData\Local\Temp/ipykernel_26280/3568192099.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resale_1_nulls['street'] = resale_1_nulls['street'].str.replace(pat="CTRL", repl="CENTRAL",case=False,regex=False)


In [10]:
#try to merge again
resale_1_nulls_fixed = resale_1_nulls.drop(columns=['Postcode'])
resale_1_nulls_fixed = resale_1_nulls_fixed.merge(postcode_lookup, how='left', on=['blk_no','street'])
resale_1_nulls_fixed.head()

,flat_type,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode
0,3 ROOM,3,SAINT GEORGE'S RD,13 TO 15,59.0,286000.0,320003
1,3 ROOM,3,SAINT GEORGE'S RD,07 TO 09,59.0,320000.0,320003
2,5 ROOM,21,SAINT GEORGE'S RD,07 TO 09,118.0,599000.0,321021
3,4 ROOM,216,CHOA CHU KANG CENTRAL,01 TO 03,108.0,345000.0,680216
4,3 ROOM,3,SAINT GEORGE'S RD,10 TO 12,59.0,305000.0,320003


In [11]:
still_null = resale_1_nulls_fixed[resale_1_nulls_fixed['Postcode'].isnull()]
still_null

,flat_type,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode


Yay!

**Combine the original blks without nulls with the fixed blocks**

In [12]:
resale_1_notnull = resale_1[resale_1['Postcode'].notnull()]
resale_1_notnull.head()

,flat_type,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode
0,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,560406.0
1,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,560108.0
2,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,560602.0
3,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,560465.0
4,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,560601.0


In [13]:
resale_2 = resale_1_notnull.append(resale_1_nulls_fixed,ignore_index=True)
resale_2.head()

,flat_type,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode
0,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,560406.0
1,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,560108.0
2,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,560602.0
3,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,560465.0
4,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,560601.0


In [14]:
#verify no nulls
resale_2.isnull().values.any()

False

In [15]:
#check number of resale transactions is consistent
len(resale_1)==len(resale_2)

True

Great job!

### Check % of resi-blks that have at least 1 resale transaction in past 5 years

In [16]:
nblks_haveresale = resi_blks[resi_blks['Postcode'].isin(resale_2['Postcode'])]
nblks_haveresale.head()

,blk_no,street,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,Postcode
0,1,BEACH RD,0,1,138,1,2,0,0,0,190001
1,1,BEDOK STH AVE 1,0,0,204,0,2,0,0,0,460001
2,1,CHAI CHEE RD,0,0,0,10,92,0,0,0,461001
3,1,CHANGI VILLAGE RD,0,0,54,0,1,0,0,0,500001
4,1,DELTA AVE,0,0,0,0,96,0,0,0,160001


In [17]:
pctblks_haveresale = len(nblks_haveresale)/len(resi_blks)*100
pctblks_haveresale

92.44327390599676

Over 92% of blocks have at least 1 resale transaction in the past 5 years.

## Break down resi-blocks by flat type present in each block

In [18]:
resi_blks.head()

,blk_no,street,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,Postcode
0,1,BEACH RD,0,1,138,1,2,0,0,0,190001
1,1,BEDOK STH AVE 1,0,0,204,0,2,0,0,0,460001
2,1,CHAI CHEE RD,0,0,0,10,92,0,0,0,461001
3,1,CHANGI VILLAGE RD,0,0,54,0,1,0,0,0,500001
4,1,DELTA AVE,0,0,0,0,96,0,0,0,160001


In [19]:
resi_blks2 = resi_blks.copy()
resi_blks2 = resi_blks2.melt(id_vars=['Postcode','blk_no','street'],value_vars=['1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold', 'studio_apartment_sold'],
                             var_name='flat_type',value_name='n_units')

resi_blks2.sort_values(['Postcode','blk_no','street','flat_type'],inplace=True)

#keep only flat types present in each block
resi_blks2 = resi_blks2[resi_blks2['n_units']>0]
resi_blks2.head(10)

,Postcode,blk_no,street,flat_type,n_units
24552,50004,4,SAGO LANE,3room_sold,248
44296,50004,4,SAGO LANE,5room_sold,2
13879,50032,32,NEW MKT RD,2room_sold,85
23751,50032,32,NEW MKT RD,3room_sold,38
33623,50032,32,NEW MKT RD,4room_sold,3
24097,50034,34,UPP CROSS ST,3room_sold,310
43841,50034,34,UPP CROSS ST,5room_sold,5
26408,50532,532,UPP CROSS ST,3room_sold,116
46152,50532,532,UPP CROSS ST,5room_sold,2
26418,50533,533,UPP CROSS ST,3room_sold,124


**reformat to facilitate merging**

In [20]:
resi_blks2['flat_type'].value_counts()

4room_sold               7541
5room_sold               5388
3room_sold               3219
exec_sold                1703
2room_sold                576
studio_apartment_sold     132
multigen_sold               8
1room_sold                  1
Name: flat_type, dtype: int64

In [21]:
resale_2['flat_type'].value_counts()

4 ROOM              45072
5 ROOM              27483
3 ROOM              25372
EXECUTIVE            8644
2 ROOM               1611
MULTI-GENERATION       52
1 ROOM                 48
Name: flat_type, dtype: int64

In [22]:
#translation table for flat type
translate_resiblk = pd.DataFrame.from_dict({'flat_type':['4room_sold','5room_sold','3room_sold','exec_sold',
                                            '2room_sold','studio_apartment_sold','multigen_sold','1room_sold'],
                               'translation': ['4 Rm','5 Rm','3 Rm','Executive', '1 Rm, 2 Rm, Studio Apt',
                                               '1 Rm, 2 Rm, Studio Apt','Multi-Gen','1 Rm, 2 Rm, Studio Apt']})

translate_resiblk

,flat_type,translation
0,4room_sold,4 Rm
1,5room_sold,5 Rm
2,3room_sold,3 Rm
3,exec_sold,Executive
4,2room_sold,"1 Rm, 2 Rm, Studio Apt"
5,studio_apartment_sold,"1 Rm, 2 Rm, Studio Apt"
6,multigen_sold,Multi-Gen
7,1room_sold,"1 Rm, 2 Rm, Studio Apt"


In [23]:
translate_resale = pd.DataFrame.from_dict({'flat_type':['4 ROOM','5 ROOM','3 ROOM','EXECUTIVE',
                                            '2 ROOM','MULTI-GENERATION','1 ROOM'],
                               'translation': ['4 Rm','5 Rm','3 Rm','Executive', '1 Rm, 2 Rm, Studio Apt',
                                               'Multi-Gen','1 Rm, 2 Rm, Studio Apt']})

translate_resale

,flat_type,translation
0,4 ROOM,4 Rm
1,5 ROOM,5 Rm
2,3 ROOM,3 Rm
3,EXECUTIVE,Executive
4,2 ROOM,"1 Rm, 2 Rm, Studio Apt"
5,MULTI-GENERATION,Multi-Gen
6,1 ROOM,"1 Rm, 2 Rm, Studio Apt"


In [24]:
resi_blks3 = resi_blks2.merge(translate_resiblk,how='left',on='flat_type').drop(columns=['flat_type']).rename(columns={'translation':'flat_type'})
resi_blks3.head()

,Postcode,blk_no,street,n_units,flat_type
0,50004,4,SAGO LANE,248,3 Rm
1,50004,4,SAGO LANE,2,5 Rm
2,50032,32,NEW MKT RD,85,"1 Rm, 2 Rm, Studio Apt"
3,50032,32,NEW MKT RD,38,3 Rm
4,50032,32,NEW MKT RD,3,4 Rm


In [25]:
#check all matched
resi_blks3.isnull().values.any()

False

In [26]:
resale_3 = resale_2.merge(translate_resale,how='left',on='flat_type').drop(columns=['flat_type']).rename(columns={'translation':'flat_type'})
resale_3.head()

,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode,flat_type
0,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,560406.0,"1 Rm, 2 Rm, Studio Apt"
1,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,560108.0,3 Rm
2,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,560602.0,3 Rm
3,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,560465.0,3 Rm
4,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,560601.0,3 Rm


In [27]:
#check all matched
resale_3.isnull().values.any()

False

## Match median resale price for each flat type in each block
- this involves averaging out between flats in the same floor level band, as well as between bands
- basically we want to see what proporation of blk-flat type combinations have resale data available

In [28]:
resale_4 = resale_3.drop(columns='storey_range').groupby(['Postcode','blk_no','street','flat_type'],as_index=False).median()

resale_4.head()

,Postcode,blk_no,street,flat_type,floor_area_sqm,resale_price
0,50004.0,4,SAGO LANE,3 Rm,67.0,400000.0
1,50032.0,32,NEW MKT RD,"1 Rm, 2 Rm, Studio Apt",52.0,313000.0
2,50032.0,32,NEW MKT RD,3 Rm,66.0,388000.0
3,50034.0,34,UPP CROSS ST,3 Rm,65.0,380000.0
4,50532.0,532,UPP CROSS ST,3 Rm,60.0,450000.0


In [29]:
#merge this with the full list of resi_blks
resi_blks4 = resi_blks3.merge(resale_4,how='left',on=['Postcode','blk_no','street','flat_type'])

resi_blks4.head()

,Postcode,blk_no,street,n_units,flat_type,floor_area_sqm,resale_price
0,50004,4,SAGO LANE,248,3 Rm,67.0,400000.0
1,50004,4,SAGO LANE,2,5 Rm,NaN,NaN
2,50032,32,NEW MKT RD,85,"1 Rm, 2 Rm, Studio Apt",52.0,313000.0
3,50032,32,NEW MKT RD,38,3 Rm,66.0,388000.0
4,50032,32,NEW MKT RD,3,4 Rm,NaN,NaN


In [30]:
n_matched = len(resi_blks4[resi_blks4['resale_price'].notnull()])

pct_matched=n_matched/len(resi_blks4)*100

In [31]:
pct_matched

80.80030159414045

Over 80% match, which is decent i think. Reasonable to say that not all blocks would have had resale transactions for all their flat types within the past 5 years

## save files for tableau

**individual resale transactions**

In [32]:
resale_3.head()

,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode,flat_type
0,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,560406.0,"1 Rm, 2 Rm, Studio Apt"
1,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,560108.0,3 Rm
2,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,560602.0,3 Rm
3,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,560465.0,3 Rm
4,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,560601.0,3 Rm


In [33]:
#convert postcode to integer. previously was float because of the nulls
resale_3['Postcode'] = resale_3['Postcode'].apply(int)

resale_3.head()

,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode,flat_type
0,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,560406,"1 Rm, 2 Rm, Studio Apt"
1,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,560108,3 Rm
2,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,560602,3 Rm
3,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,560465,3 Rm
4,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,560601,3 Rm


In [34]:
#calculate resale price per square metre
resale_3['resale_price_psm'] = resale_3['resale_price']/resale_3['floor_area_sqm']
resale_3.head()

,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode,flat_type,resale_price_psm
0,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,560406,"1 Rm, 2 Rm, Studio Apt",5272.727273
1,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,560108,3 Rm,3731.343284
2,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,560602,3 Rm,3910.447761
3,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,560465,3 Rm,3897.058824
4,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,560601,3 Rm,3955.223881


In [35]:
resale_3.isnull().values.any()

False

In [36]:
resale_3.sort_values(['Postcode','blk_no','street','flat_type'], inplace=True)

resale_3.head(50)

,blk_no,street,storey_range,floor_area_sqm,resale_price,Postcode,flat_type,resale_price_psm
1431,4,SAGO LANE,16 TO 18,67.0,400000.0,50004,3 Rm,5970.149254
4611,4,SAGO LANE,13 TO 15,67.0,410000.0,50004,3 Rm,6119.402985
4612,4,SAGO LANE,10 TO 12,67.0,410000.0,50004,3 Rm,6119.402985
4613,4,SAGO LANE,07 TO 09,67.0,410000.0,50004,3 Rm,6119.402985
4614,4,SAGO LANE,19 TO 21,67.0,425000.0,50004,3 Rm,6343.283582
19250,4,SAGO LANE,19 TO 21,67.0,410000.0,50004,3 Rm,6119.402985
23168,4,SAGO LANE,04 TO 06,137.0,560000.0,50004,3 Rm,4087.591241
26808,4,SAGO LANE,10 TO 12,67.0,387000.0,50004,3 Rm,5776.119403
35184,4,SAGO LANE,10 TO 12,67.0,388000.0,50004,3 Rm,5791.044776
37159,4,SAGO LANE,19 TO 21,67.0,393888.0,50004,3 Rm,5878.925373


In [37]:
**STOP DO NOT OVERWRITE!**
resale_3.to_csv("Clean Datasets/resale_Jan2017.csv",index=False)

**median by block**

In [38]:
resale_3_medbyblock = resale_3.drop(columns=['storey_range','floor_area_sqm','resale_price']).groupby(['Postcode','blk_no','street','flat_type'],
                                                                   as_index=False).median()

resale_3_medbyblock = resale_3_medbyblock.pivot(index=['Postcode','blk_no','street'],
                                                columns='flat_type',values='resale_price_psm').reset_index()

resale_3_medbyblock.head()

flat_type,Postcode,blk_no,street,"1 Rm, 2 Rm, Studio Apt",3 Rm,4 Rm,5 Rm,Executive,Multi-Gen
0,50004,4,SAGO LANE,NaN,5970.149254,NaN,NaN,NaN,NaN
1,50032,32,NEW MKT RD,6019.230769,5878.787879,NaN,NaN,NaN,NaN
2,50034,34,UPP CROSS ST,NaN,5846.153846,NaN,NaN,NaN,NaN
3,50532,532,UPP CROSS ST,NaN,7470.588235,NaN,NaN,NaN,NaN
4,50533,533,UPP CROSS ST,NaN,7600.000000,NaN,NaN,NaN,NaN


In [39]:
**STOP DO NOT OVERWRITE!**
resale_3_medbyblock.to_csv("Clean Datasets/resale_Jan2017_BlkMedians.csv",index=False)